In [1]:
%%file demo_2d.py

from __future__ import print_function, division
import os
import sys, petsc4py
petsc4py.init(sys.argv)
import mpi4py.MPI as mpi
from petsc4py import PETSc
import numpy as np
from GenEO import *

def rhs(coords, rhs):
    n = rhs.shape
    rhs[..., 1] = -9.81

OptDB = PETSc.Options()
Lx = OptDB.getInt('Lx', 10)
Ly = OptDB.getInt('Ly', 1)
n  = OptDB.getInt('n', 16)
nx = OptDB.getInt('nx', Lx*n)
ny = OptDB.getInt('ny', Ly*n)
E1 = OptDB.getReal('E1', 10**6)
E2 = OptDB.getReal('E2', 1)
nu1 = OptDB.getReal('nu1', 0.4)
nu2 = OptDB.getReal('nu2', 0.4)

hx = Lx/(nx - 1)
hy = Ly/(ny - 1)

da = PETSc.DMDA().create([nx, ny], dof=2, stencil_width=1)
da.setUniformCoordinates(xmax=Lx, ymax=Ly)
da.setMatType(PETSc.Mat.Type.IS)
da.setFieldName(0, 'u')
da.setFieldName(1, 'v')

path = './output_2d/'
if mpi.COMM_WORLD.rank == 0:
    if not os.path.exists(path):
        os.mkdir(path)
    else:
        os.system('rm {}/*.vts'.format(path))

def lame_coeff(x, y, v1, v2):
    output = np.empty(x.shape)
    mask = np.logical_or(np.logical_and(.2<=y, y<=.4),np.logical_and(.6<=y, y<=.8))
    output[mask] = v1
    output[np.logical_not(mask)] = v2
    return output

# non constant Young's modulus and Poisson's ratio 
E = buildCellArrayWithFunction(da, lame_coeff, (E1,E2))
nu = buildCellArrayWithFunction(da, lame_coeff, (nu1,nu2))

lamb = (nu*E)/((1+nu)*(1-2*nu)) 
mu = .5*E/(1+nu)

class callback:
    def __init__(self, da):
        self.da = da
        ranges = da.getRanges()
        ghost_ranges = da.getGhostRanges()
        
        self.slices = []
        for r, gr in zip(ranges, ghost_ranges):
            self.slices.append(slice(gr[0], r[1]))
        self.slices = tuple(self.slices)

        self.it = 0

    def __call__(self, locals):
        pyKSP = locals['self']
        proj = pyKSP.mpc.proj

        if self.it == 0:
            work, _ = proj.A.getVecs()
            for i, vec in enumerate(proj.coarse_vecs):
                if vec:
                    proj.workl = vec.copy()
                else:
                    proj.workl.set(0.)
                work.set(0)
                proj.scatter_l2g(proj.workl, work, PETSc.InsertMode.ADD_VALUES)

                viewer = PETSc.Viewer().createVTK(path + 'coarse_vec_{}.vts'.format(i), 'w', comm = PETSc.COMM_WORLD)
                tmp = self.da.createGlobalVec()
                tmpl_a = self.da.getVecArray(tmp)
                work_a = self.da.getVecArray(work)
                tmpl_a[:] = work_a[:]
                tmp.view(viewer)
                viewer.destroy()
            self.it += 1


x = da.createGlobalVec()
b = buildRHS(da, [hx, hy], rhs)
A = buildElasticityMatrix(da, [hx, hy], lamb, mu)
A.assemble()
bcApplyWest(da, A, b)

#Setup the preconditioner (or multipreconditioner) and the coarse space
pcbnn = PCBNN(A)

# Set initial guess
x.setRandom()
xnorm = b.dot(x)/x.dot(A*x)
x *= xnorm

ksp = PETSc.KSP().create()
ksp.setOperators(A)
ksp.setType(ksp.Type.PYTHON)
pyKSP = KSP_AMPCG(pcbnn)
pyKSP.callback = callback(da)
ksp.setPythonContext(pyKSP)
ksp.setFromOptions()
ksp.setInitialGuessNonzero(True)

ksp.solve(b, x)

viewer = PETSc.Viewer().createVTK(path + 'solution_2d.vts', 'w', comm = PETSc.COMM_WORLD)
x.view(viewer)


lamb_petsc = da.createGlobalVec()
lamb_a = da.getVecArray(lamb_petsc)
coords = da.getCoordinates()
coords_a = da.getVecArray(coords)
E = lame_coeff(coords_a[..., 0], coords_a[..., 1], E1, E2)
nu = lame_coeff(coords_a[..., 0], coords_a[..., 1], nu1, nu2)

lamb_a[..., 0] = (nu*E)/((1+nu)*(1-2*nu)) 
lamb_a[..., 1] = mpi.COMM_WORLD.rank
lamb_petsc.view(viewer)

viewer.destroy()

Overwriting demo_2d.py


In [2]:
!mpiexec -np 4 python demo_2d.py -AMPCG_verbose -ksp_monitor -PCBNN_verbose

Subdomain number 0 contributes 0 coarse vectors as zero energy modes of local solver
Subdomain number 2 contributes 3 coarse vectors as zero energy modes of local solver
Subdomain number 1 contributes 3 coarse vectors as zero energy modes of local solver
Subdomain number 3 contributes 3 coarse vectors as zero energy modes of local solver
GenEO eigenvalue number 0 for lambdamax in subdomain 2: (1.4945905664998919e-06+0j)
GenEO eigenvalue number 1 for lambdamax in subdomain 2: (1.3826304536173795e-05+0j)
GenEO eigenvalue number 2 for lambdamax in subdomain 2: (4.5054862698830293e-05+0j)
GenEO eigenvalue number 3 for lambdamax in subdomain 2: (0.05405868128473588+0j)
GenEO eigenvalue number 4 for lambdamax in subdomain 2: (0.054059009027751584+0j)
GenEO eigenvalue number 5 for lambdamax in subdomain 2: (0.05666304812554981+0j)
GenEO eigenvalue number 6 for lambdamax in subdomain 2: (0.05707962286854576+0j)
GenEO eigenvalue number 7 for lambdamax in subdomain 2: (0.15901212849796265+0j)
Ge

In [3]:
from plot import plot_solution

In [4]:
plot_solution('output_2d', 'solution_2d.vts')

HBox(children=(Renderer(camera=PerspectiveCamera(position=(0.0, 0.0, 13.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)), controls=[OrbitControls(controlling=PerspectiveCamera(position=(0.0, 0.0, 13.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)), enableRotate=False)], scene=Scene(background='black', children=(Mesh(geometry=BufferGeometry(attributes={'position': <BufferAttribute shape=(14310, 3), dtype=float32>, 'color': <BufferAttribute shape=(4770, 3, 3), dtype=float32>}), material=MeshLambertMaterial(alphaMap=None, aoMap=None, color='0xF5F5F5', emissiveMap=None, envMap=None, lightMap=None, map=None, opacity=0.5, side='DoubleSide', specularMap=None, transparent=True, vertexColors='VertexColors'), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)), LineSegments(geometry=BufferGeometry(attributes={'position': <BufferAttribute shape=(19080, 3), dtype=float32>}), material=LineBasicMaterial(color='0xffffff'), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)), AmbientLight(quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0))), fog=None, overrideMaterial=None, position=(-5.192105760650471, 2.9941447310517106, -0.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)), shadowMap=WebGLShadowMap()), VBox(children=(Dropdown(description='Fields', options=OrderedDict([('u', 0), ('v', 1), ('lambda', 2), ('rank', 3)]), value=0), Dropdown(description='domains', options=OrderedDict([('all', -1), ('domain 0', 0), ('domain 1', 1), ('domain 2', 2), ('domain 3', 3)]), value=-1), Dropdown(description='coarse vecs', options=OrderedDict([('none', -1), ('coarse vec 0', 0), ('coarse vec 1', 1), ('coarse vec 2', 2), ('coarse vec 3', 3), ('coarse vec 4', 4), ('coarse vec 5', 5), ('coarse vec 6', 6), ('coarse vec 7', 7), ('coarse vec 8', 8), ('coarse vec 9', 9), ('coarse vec 10', 10), ('coarse vec 11', 11), ('coarse vec 12', 12), ('coarse vec 13', 13), ('coarse vec 14', 14), ('coarse vec 15', 15), ('coarse vec 16', 16), ('coarse vec 17', 17), ('coarse vec 18', 18), ('coarse vec 19', 19), ('coarse vec 20', 20), ('coarse vec 21', 21), ('coarse vec 22', 22), ('coarse vec 23', 23), ('coarse vec 24', 24), ('coarse vec 25', 25), ('coarse vec 26', 26), ('coarse vec 27', 27), ('coarse vec 28', 28), ('coarse vec 29', 29), ('coarse vec 30', 30), ('coarse vec 31', 31), ('coarse vec 32', 32), ('coarse vec 33', 33), ('coarse vec 34', 34), ('coarse vec 35', 35)]), value=-1), Checkbox(value=True, description='Show displacement'), Checkbox(value=True, description='Show mesh'), IntSlider(value=100, continuous_update=False, description='Scale factor for coarse vec displacement', max=1000, min=1)))))